In [1]:
import numpy as np

## Input Parameters

In [2]:
# geometry
L = 1.0
w = 0.05
h = 0.1

# mesh
shape = (100, 10, 5)

# material
E = 1000
nu = 0.2

# load
F = 100

## Analytic Bernoulli Solution

In [3]:
EI = E * w * h**3 / 12
UY_Bernoulli = -F * L**3 / (3 * EI)

## Finite Element Solution

In [4]:
# Hooke model
A = np.array([
    [1, nu, nu, 0, 0, 0], 
    [nu, 1, nu, 0, 0, 0],
    [nu, nu, 1, 0, 0, 0], 
    [0., 0, 0, (1-nu)/2, 0, 0],
    [0., 0, 0, 0, (1-nu)/2, 0],
    [0., 0, 0, 0, 0, (1-nu)/2]]) * (E /(1-nu**2))

In [5]:
from polymesh import PolyData
from polymesh.grid import grid
from polymesh.cells import H8

coords, topo = grid(size=(L, h, w), shape=shape, eshape='H8')
mesh = PolyData(coords=coords, topo=topo, celltype=H8)

In [6]:
# essential boundary conditions
cond_ebc = np.abs(coords[:, 0]) <= 1e-5
ebcinds = np.where(cond_ebc)[0]
fixity = np.zeros((coords.shape[0], 3), dtype=bool)
fixity[ebcinds, :] = True

In [7]:
# natural boundary conditions
target = np.array([L, h/2, 0])
nbcinds = [mesh.index_of_closest_point(target)]
loads = np.zeros((coords.shape[0], 3))
loads[nbcinds, 1] = -F

In [8]:
from sigmaepsilon.solid.fem import FemMesh, Structure
from polymesh.cells import H8
mesh = FemMesh(coords=coords, topo=topo, celltype=H8, 
               fixity=fixity, loads=loads, model=A)
structure = Structure(mesh=mesh)

In [9]:
structure.linsolve()
structure.summary

{'preproc': {'sparsify': False},
 'proc': {'time [ms]': 356.0004234313965,
  'N': 19998,
  'use_umfpack': True,
  'permc_spec': 'COLAMD',
  'solver': 'pardiso'},
 'postproc': {},
 'number of elements': 5000,
 'number of nodes': 6666,
 'dofs per node': 3}

In [10]:
UY_FEM = structure.pointdata.dofsol.to_numpy()[nbcinds[0]][1]

## Comparison

In [11]:
print("Analytic Bernoulli Solution : {}".format(UY_Bernoulli))
print("FEM Solution : {}".format(UY_FEM))
diff = 100 * (UY_FEM - UY_Bernoulli) / UY_Bernoulli
print("Difference : {} %".format(diff))
structure.plot()

Analytic Bernoulli Solution : -7999.999999999998
FEM Solution : -8413.868762297314
Difference : 5.173359528716447 %
